In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [107]:
pandf = fdr.DataReader("KS11","2009-01-01","2019-01-01")


In [3]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [124]:
#역사적변동성
import math
list=[] #로그수익률
for x in range(pandf.shape[0]-1):
    list.append(math.log(pandf["Close"][x+1] / pandf["Close"][x]))
average = sum(list)/len(list) # 로그수익률의 평균

list2=[] # 로그수익률의 편차의 제곱
for x in range(pandf.shape[0]-1):
    list2.append((average - (math.log(pandf["Close"][x+1] / pandf["Close"][x]))) **2)
average2 = sum(list2)/len(list2) #편차의 제곱의 평균

V=math.sqrt(average2 * 252) # 역사적변동성

list3=[] # 그냥 표준편차
for x in pandf["Change"]:
    x=abs(x)
    list3.append(math.sqrt(x)/V) # 표준편차/역사적변동성
    
df=pandf.assign(V=list3)

In [125]:

df=get_time(df)
df=get_stochastic(df)
df=get_MA(df)
# convert nparray

#y=np.where(df["Change"].shift(-1).dropna()>0,1,0)  # 값과값비교
nparr = df.values
nparr.astype('float32')
print(nparr)

[[1169.95       1176.86       1176.86       ...   85.79334492
  1104.68038462 1134.385     ]
 [1161.81       1180.39       1186.91       ...   87.42346904
  1103.58923077 1134.46980769]
 [1170.94       1165.03       1180.38       ...   88.69249321
  1103.24692308 1134.41923077]
 ...
 [2028.01       2028.81       2037.83       ...   21.65334887
  2078.89038462 2083.74788462]
 [2028.44       2032.09       2035.57       ...   19.78398169
  2076.80807692 2081.50403846]
 [2041.04       2036.7        2046.97       ...   20.81083906
  2075.44230769 2079.5025    ]]


In [137]:
df

,Close,Open,High,Low,Volume,Change,V,time,kdj_k,kdj_d,kdj_j,MA_26,MA_52
Date,,,,,,,,,,,,,
2009-03-18,1169.95,1176.86,1176.86,1157.11,520430000.0,0.0052,0.444594,0.206144,96.248032,90.042134,85.793345,1104.680385,1134.385000
2009-03-19,1161.81,1180.39,1186.91,1158.19,532580000.0,-0.0070,0.515835,0.210186,87.076511,89.053593,87.423469,1103.589231,1134.469808
2009-03-20,1170.94,1165.03,1180.38,1163.59,374670000.0,0.0079,0.547994,0.214228,91.777366,89.961517,88.692493,1103.246923,1134.419231
2009-03-23,1199.50,1179.60,1201.23,1179.13,461580000.0,0.0244,0.963068,0.218270,99.170423,93.031153,90.861823,1103.518462,1134.519615
2009-03-24,1221.70,1225.94,1225.94,1205.08,512520000.0,0.0185,0.838586,0.222312,98.048511,94.703605,92.782714,1105.296538,1134.395192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,2061.49,2052.70,2061.51,2049.76,311390000.0,0.0007,0.163121,9.979790,19.565450,24.885223,27.398510,2083.118846,2087.757500
2018-12-24,2055.01,2050.38,2059.94,2046.18,285280000.0,-0.0031,0.343276,9.983832,13.764143,21.178196,24.288353,2081.680769,2086.321731
2018-12-26,2028.01,2028.81,2037.83,2014.28,321500000.0,-0.0131,0.705663,9.987874,14.698640,19.018344,21.653349,2078.890385,2083.747885


In [127]:
# normalization
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)

In [14]:
X=nptf[:-1] #값과값비교

In [128]:

# 값과 이평선 비교하기 ----------------

MA_26=df["Close"].rolling(26).mean()
y_before=MA_26.dropna()
df_before=df["Close"][25:]
y=np.where(df_before.shift(-1)>y_before.shift(-1),1,0)
X=nptf[25:]

In [129]:
# split train, test
train_size = int(len(nptf) * 0.7)
y_train2=y[:train_size]
y_test2=y[train_size:]
X_train2 = X[:train_size]
X_test2 = X[train_size:]



In [130]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train2, (X_train2.shape[0], 1, X_train2.shape[1]))
X_test = np.reshape(X_test2, (X_test2.shape[0], 1, X_test2.shape[1]))
y_train=y_train2
y_test=y_test2


In [140]:
# simple lstm network learning
model = Sequential()
model.add(LSTM(36, input_shape=(1, 13)))
for i in range(5):
    model.add(Dense(36,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, batch_size=16,validation_split=0.1)
model.evaluate(X_test,y_test)

Train on 1526 samples, validate on 170 samples
Epoch 1/10
1526/1526 [==============================] - 2s 1ms/step - loss: 0.6773 - acc: 0.5885 - val_loss: 0.7310 - val_acc: 0.3765
Epoch 2/10
1526/1526 [==============================] - 0s 270us/step - loss: 0.6799 - acc: 0.5885 - val_loss: 0.7433 - val_acc: 0.3765
Epoch 3/10
1526/1526 [==============================] - 0s 229us/step - loss: 0.5939 - acc: 0.6691 - val_loss: 0.4875 - val_acc: 0.8412
Epoch 4/10
1526/1526 [==============================] - 0s 248us/step - loss: 0.3942 - acc: 0.8552 - val_loss: 0.3992 - val_acc: 0.8412
Epoch 5/10
1526/1526 [==============================] - 0s 248us/step - loss: 0.3687 - acc: 0.8591 - val_loss: 0.4218 - val_acc: 0.8235
Epoch 6/10
1526/1526 [==============================] - 0s 242us/step - loss: 0.3551 - acc: 0.8539 - val_loss: 0.4120 - val_acc: 0.8235
Epoch 7/10
1526/1526 [==============================] - 0s 242us/step - loss: 0.3380 - acc: 0.8644 - val_loss: 0.4018 - val_acc: 0.8294
Epo

[0.3560661707508598, 0.8475783467292786]

In [141]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,np.where(model.predict(X_test)>0.5,1,0))

0.8475783475783476

In [87]:
model.predict(X_test)

array([[0.59089255],
       [0.5908443 ],
       [0.59091526],
       [0.59089494],
       [0.5908968 ],
       [0.5909186 ],
       [0.5908744 ],
       [0.59087527],
       [0.5909183 ],
       [0.59087586],
       [0.590822  ],
       [0.5908731 ],
       [0.5909016 ],
       [0.5909233 ],
       [0.59087604],
       [0.5909101 ],
       [0.5908844 ],
       [0.5908661 ],
       [0.5908756 ],
       [0.59086674],
       [0.5908814 ],
       [0.59087044],
       [0.59086347],
       [0.59091645],
       [0.5909127 ],
       [0.5908743 ],
       [0.5908863 ],
       [0.59087175],
       [0.590894  ],
       [0.5909056 ],
       [0.5908779 ],
       [0.59088767],
       [0.59086394],
       [0.59087944],
       [0.59088916],
       [0.5908985 ],
       [0.59087706],
       [0.5909015 ],
       [0.5908879 ],
       [0.5908729 ],
       [0.5909075 ],
       [0.5908975 ],
       [0.5909084 ],
       [0.59089875],
       [0.59087515],
       [0.59086776],
       [0.5908946 ],
       [0.590

In [395]:
MA_26=df["Close"].rolling(26).mean()
MA_26=MA_26.dropna()
MA_26.shape


(713,)

In [396]:
pandf.shape

(789, 6)

In [397]:
MA_26.shape

(713,)

In [398]:
X_train2.shape

(516, 12)